In [1]:
import numpy as np

import torch
import pyro.distributions as dist

import pyro
import pyro.poutine as poutine
from pyro.infer.mcmc import MCMC, NUTS, HMC
from pyro.infer import EmpiricalMarginal

import arviz as az

In [2]:
az.__version__

'0.3.1'

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

# Data

In [13]:
np.random.seed(123)

# True parameter values
alpha, sigma = 1.0, 1.0
beta = torch.tensor(np.array([1.0, 2.5]))

# Size of dataset
size = 100

# Predictor variable
X1 = torch.tensor(np.random.randn(size))
X2 = torch.tensor(np.random.randn(size) * 0.2)

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + torch.tensor(np.random.randn(size)) * sigma

# Model

In [14]:
#     # Priors for unknown model parameters
#     alpha = pm.Normal('alpha', mu=0, sigma=10)
#     beta = pm.Normal('beta', mu=0, sigma=10, shape=2)
#     sigma = pm.HalfNormal('sigma', sigma=1)

#     # Expected value of outcome
#     mu = alpha + beta[0]*X1 + beta[1]*X2

#     # Likelihood (sampling distribution) of observations
#     Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=Y)

In [15]:
def model(x1, x2):
    
    alpha = pyro.sample('alpha', dist.Normal(0, 10))
    beta = pyro.sample('beta', dist.Normal(torch.zeros(2), torch.Tensor([10, 10])))
    sigma = pyro.sample('sigma', dist.HalfNormal(scale=1))
    
#     print(alpha)
#     print(beta)
#     print(sigma)
    
    mu = alpha + beta[0] * x1 + beta[1] * x2
    
#     print(mu)
    
    y = pyro.sample('y', dist.Normal(mu, sigma))
    return y

def conditioned_model(model, x1, x2, y):
    return poutine.condition(model, data={'y': y})(x1, x2)

In [19]:
class Model(object):
    
    def __init__(self, *args, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def model_fn(self, x1, x2):
        alpha = pyro.sample('alpha', dist.Normal(0, 10))
        beta = pyro.sample('beta', dist.Normal(torch.zeros(2), torch.Tensor([10, 10])))
        sigma = pyro.sample('sigma', dist.HalfNormal(scale=1)) 
        
        mu = alpha + beta[0] * x1 + beta[1] * x2
    
        y = pyro.sample('y', dist.Normal(mu, sigma))
        return y
    
    def observe(self, *args, **kwargs):
#         print(**kwargs)
        for key, value in kwargs.items():
            print("{} : {}".format(key, value.shape))
            setattr(self, key, value)
            self.conditioned = poutine.condition(self.model_fn, data={key: value})
            return
    
    def sample(self):
        print(self.conditioned)
        nuts_kernel = NUTS(self.conditioned, adapt_step_size=True)
        print(nuts_kernel)
        posterior = MCMC(nuts_kernel, num_samples=500, warmup_steps=100).run(self.x1, self.x2)
        return posterior

In [20]:
model = Model(x1=X1, x2=X2)
model.observe(y=Y)
trace = model.sample()

y : torch.Size([100])
<function Messenger.__call__.<locals>._wraps at 0x1297250d0>


In [21]:
trace_data = az.from_pyro(trace)

In [22]:
trace_data.observed_data

<xarray.Dataset>
Dimensions:  (chain: 1, draw: 500, y_dim_0: 100)
Coordinates:
  * chain    (chain) int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * y_dim_0  (y_dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    y        (chain, draw, y_dim_0) float64 0.9387 0.4103 3.84 ... 1.369 1.332
Attributes:
    created_at:         2019-01-10T16:11:30.123758
    inference_library:  pyro

In [23]:
trace_data.posterior

<xarray.Dataset>
Dimensions:     (beta_dim_0: 2, chain: 1, draw: 500)
Coordinates:
  * chain       (chain) int64 0
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 492 493 494 495 496 497 498 499
  * beta_dim_0  (beta_dim_0) int64 0 1
Data variables:
    alpha       (chain, draw) float64 0.865 0.7748 1.028 ... 0.9989 0.8078 1.03
    beta        (chain, draw, beta_dim_0) float64 0.9834 2.09 ... 1.191 2.285
    sigma       (chain, draw) float64 1.128 1.147 1.143 ... 0.9887 0.9876 1.01
Attributes:
    created_at:         2019-01-10T16:11:30.031872
    inference_library:  pyro

# Original

In [5]:
def model(x1, x2):
    
    alpha = pyro.sample('alpha', dist.Normal(0, 10))
    beta = pyro.sample('beta', dist.Normal(torch.zeros(2), torch.Tensor([10, 10])))
    sigma = pyro.sample('sigma', dist.HalfNormal(scale=1))
    
    mu = alpha + beta[0] * x1 + beta[1] * x2
    
    y = pyro.sample('y', dist.Normal(mu, sigma))
    return y

def conditioned_model(model, x1, x2, y):
    return poutine.condition(model, data={'y': y})(x1, x2)

In [6]:
# nuts_kernel = NUTS(model, adapt_step_size=True)
nuts_kernel = NUTS(conditioned_model, adapt_step_size=True)
posterior = MCMC(nuts_kernel, num_samples=500, warmup_steps=100).run(model, X1, X2, Y)

In [7]:
# {'alpha': array(0.90907964),
#  'beta': array([0.9514399 , 2.61452795]),
#  'sigma_log__': array(-0.03492212),
#  'sigma': array(0.96568062)}

In [8]:
pyro_data = az.from_pyro(posterior)

In [9]:
pyro_data

Inference data with groups:
	> posterior
	> observed_data

In [10]:
az.summary(pyro_data)

,mean,sd,mc error,hpd 3%,hpd 97%
alpha,0.90,0.10,0.01,0.72,1.11
beta[0],0.95,0.08,0.00,0.80,1.11
beta[1],2.64,0.48,0.03,1.83,3.49
sigma,0.99,0.07,0.01,0.85,1.13


In [11]:
az.plot_trace(pyro_data);

In [12]:
az.plot_posterior(pyro_data);